In [ ]:
import pandas as pd

In [6]:
"""
Weight free throw effect based on how many shots taken
"""
def ft_stat(player_percentage, player_attempts):
    league_percentage = .85
    return (player_percentage - league_percentage) * player_attempts

"""
Weight field goal effect based on how many shots taken
"""
def fg_stat(player_percentage, player_attempts):
    league_percentage = .47
    return (player_percentage - league_percentage) * player_attempts

"""
Turn players into list of dicts
"""
def create_player_list(players):
    fg, ft = 0, 0
    players_stats = {} #stats for a single player
    new_players = [] #list of dicts that holds player and weighted ft and fg
    for i in range(len(players)):
        if i % 2 == 0:
            player_stats = players[i][1].copy(deep=True).to_dict()
        else:
            ft = players[i][1]["FT%"]
            fg = players[i][1]["FG%"]
            ft = ft.split("/")
            fg = fg.split("/")
            ft_attempts = ft[1][:-1]
            fg_attempts = fg[1][:-1]
            player_stats["weighted_ft"] = ft_stat(float(player_stats["FT%"]), float(ft_attempts))
            player_stats["weighted_fg"] = fg_stat(float(player_stats["FG%"]), float(fg_attempts))
            new_players.append(player_stats)
            player_stats = {}
    return new_players
"""
Get a list of all players that were drafted. TODO: Change to lg instead of lg1 during the draft
@return (list of str) - player names that were drafted
"""
def drafted():
    list = []
    for player in lg.taken_players():
        list.append(player['name'])
    return list
"""
Remove all drafted players from the dataframe
pd (pandas dataframe) - dataframe of all available players
drafted_players (List of str) - players that were drafted
"""
def remove_drafted_players(df, drafted_players):
    return df[~df.PLAYER.isin(drafted_players)]

#Turnovers should be negative to make it subtract from total_val
required_stats = {"FG%": .47, "FT%": .798, "3PM": 1.17, "PTS": 15.137, "TREB": 6.241, "AST": 3.748, "STL": 1.132, "BLK": .862, "TO": -1.7}

"""
Albert's personal rankings that takes into considerations punts
Ranking Methodology
Number of Games played * by the following
1. Value per stat
2. Remove all categories I punt
@param pd (pandas dataframe) - dataframe of all available players
@param punts(list of str) - stats to punt

@return pandas dataframe with additional column: Albert's ranking
"""
def alberts_ranking(df, punts):
    df = df.to_dict('records')
    for row in df:
        total_val = 0
        for stat in row.items():
            if stat[0] in required_stats and stat[0] not in punts:
                total_val += min(stat[1] / required_stats[stat[0]], 3)
        row['total_val'] = total_val * row['GP']
    return pd.DataFrame(df).sort_values("total_val", ascending=False)


In [3]:
from yahoo_oauth import OAuth2
import json
creds = {'consumer_key': "dj0yJmk9U25NRHowdE16MFNHJmQ9WVdrOVNuaGtSa3RaYTBVbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTFh", 'consumer_secret': 'd83900e9c38441740e701ea87bf4b437f07a501c'}
with open("sample.json", "w") as outfile:  
    json.dump(creds, outfile) 
oauth = OAuth2(None, None, from_file='sample.json')

import yahoo_fantasy_api
lg = yahoo_fantasy_api.game.Game(oauth, "nba").to_league("402.l.5116")

lg1 = yahoo_fantasy_api.game.Game(oauth, "nba").to_league("402.l.441")

[2020-12-21 16:19:50,822 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2020-12-21 16:19:50,825 DEBUG] [yahoo_oauth.oauth.handler] AUTHORIZATION URL : https://api.login.yahoo.com/oauth2/request_auth?redirect_uri=oob&response_type=code&client_id=dj0yJmk9U25NRHowdE16MFNHJmQ9WVdrOVNuaGtSa3RaYTBVbWNHbzlNQT09JnM9Y29uc3VtZXJzZWNyZXQmc3Y9MCZ4PTFh


Enter verifier : 4aeec8x


In [4]:
players = pd.read_excel('hashtag.xlsx', index_col=0)

alberts_players = players.copy(deep=True)
players = [i for i in alberts_players.iterrows()]
#Add weighted free throw and fg stats to table

players = create_player_list(players)
df = pd.DataFrame(players).sort_values("TOTAL", ascending=False)
df

remove_drafted_players(df, drafted())

,3PM,AST,BLK,FG%,FT%,GP,MPG,PLAYER,PTS,R#,STL,TO,TOTAL,TREB,weighted_fg,weighted_ft
74,0.43,2.54,0.55,0.586,0.649,60.0,31.5,Zion Williamson,25.81,75.0,0.88,2.98,2.27,7.82,1.97200,-1.68438
27,0.00,1.52,2.10,0.695,0.626,67.0,34.8,Rudy Gobert,15.32,28.0,0.71,1.93,5.85,13.70,1.87200,-1.34176
12,1.62,5.67,1.01,0.546,0.664,69.0,30.8,Giannis Antetokounmpo,29.84,13.0,1.01,3.75,8.37,13.78,1.50176,-1.86000
65,0.00,0.57,1.95,0.698,0.568,68.0,23.2,Mitchell Robinson,9.69,66.0,0.92,0.68,2.77,7.26,1.36116,-0.66552
15,0.21,1.99,1.63,0.553,0.760,67.0,32.5,Deandre Ayton,19.10,16.0,0.82,2.05,7.62,11.66,1.26160,-0.24750
56,0.42,1.58,1.06,0.597,0.766,68.0,25.2,Brandon Clarke,13.64,57.0,0.68,1.16,3.19,6.79,1.22555,-0.18648
137,0.00,1.52,0.98,0.582,0.661,64.0,26.1,Montrezl Harrell,15.85,138.0,0.53,1.52,-0.24,6.33,1.20848,-0.94311
73,0.00,1.37,1.56,0.606,0.521,59.0,29.7,Clint Capela,11.91,74.0,0.83,1.64,2.41,10.73,1.18456,-0.85211
18,0.00,5.33,1.36,0.565,0.720,65.0,34.5,Bam Adebayo,17.86,19.0,1.15,2.92,7.17,10.76,1.16755,-0.72020
82,0.01,1.40,1.44,0.642,0.662,70.0,26.4,Jarrett Allen,11.04,83.0,0.50,1.20,1.88,9.60,1.15412,-0.68432


In [7]:
ranking = alberts_ranking(df, [])
ranking

,3PM,AST,BLK,FG%,FT%,GP,MPG,PLAYER,PTS,R#,STL,TO,TOTAL,TREB,total_val,weighted_fg,weighted_ft
2,3.25,3.73,1.52,0.501,0.804,69.0,33.1,Karl-Anthony Towns,25.11,3.0,1.10,3.41,12.17,11.09,690.790024,0.52049,-0.28704
13,3.10,4.24,0.85,0.455,0.813,70.0,34.6,Jayson Tatum,24.93,14.0,1.41,2.41,8.06,7.11,655.755654,-0.29550,-0.17797
0,4.52,8.03,0.81,0.442,0.865,64.0,36.3,James Harden,34.02,1.0,1.71,4.59,16.54,6.64,654.626716,-0.60452,0.18075
1,1.32,3.24,2.33,0.503,0.847,65.0,34.8,Anthony Davis,26.61,2.0,1.52,2.53,14.62,9.41,646.589767,0.59103,-0.02580
4,3.96,7.69,0.26,0.456,0.897,67.0,35.9,Damian Lillard,28.70,5.0,1.12,2.96,11.57,4.17,620.423976,-0.27482,0.35720
12,1.62,5.67,1.01,0.546,0.664,69.0,30.8,Giannis Antetokounmpo,29.84,13.0,1.01,3.75,8.37,13.78,616.069957,1.50176,-1.86000
8,2.91,9.05,0.31,0.465,0.768,67.0,34.1,Luka Doncic,29.40,9.0,1.10,4.20,9.13,9.52,615.193838,-0.10355,-0.76998
7,3.44,9.60,0.10,0.443,0.866,68.0,34.1,Trae Young,26.84,8.0,1.01,3.91,9.55,3.50,582.860972,-0.47439,0.14448
33,2.80,6.84,0.31,0.412,0.838,67.0,35.6,Fred VanVleet,18.24,34.0,1.87,2.28,5.42,3.83,578.473352,-0.85956,-0.04596
6,1.24,7.30,0.73,0.519,0.819,70.0,32.3,Nikola Jokic,20.60,7.0,1.31,3.24,9.98,10.32,577.557381,0.74627,-0.13392
